In [1]:
from bertviz import attention, visualization
from bertviz.pytorch_pretrained_bert import BertModel, BertTokenizer
import numpy as np
import random
import torch
import h5py
random.seed(1111)

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
bert_version = 'bert-base-uncased'
model = BertModel.from_pretrained(bert_version)


In [3]:
i=0
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)
        #param.data.uniform_(-0.1, 0.1)

embeddings.word_embeddings.weight
embeddings.position_embeddings.weight
embeddings.token_type_embeddings.weight
embeddings.LayerNorm.weight
embeddings.LayerNorm.bias
encoder.layer.0.attention.self.query.weight
encoder.layer.0.attention.self.query.bias
encoder.layer.0.attention.self.key.weight
encoder.layer.0.attention.self.key.bias
encoder.layer.0.attention.self.value.weight
encoder.layer.0.attention.self.value.bias
encoder.layer.0.attention.output.dense.weight
encoder.layer.0.attention.output.dense.bias
encoder.layer.0.attention.output.LayerNorm.weight
encoder.layer.0.attention.output.LayerNorm.bias
encoder.layer.0.intermediate.dense.weight
encoder.layer.0.intermediate.dense.bias
encoder.layer.0.output.dense.weight
encoder.layer.0.output.dense.bias
encoder.layer.0.output.LayerNorm.weight
encoder.layer.0.output.LayerNorm.bias
encoder.layer.1.attention.self.query.weight
encoder.layer.1.attention.self.query.bias
encoder.layer.1.attention.self.key.weight
encoder.layer.1.attention.self.key

In [4]:

tokenizer = BertTokenizer.from_pretrained(bert_version)

filename = '/scratch/sb6416/Ling3340/extract_tree/UD_English-PUD/en_pud-ud-test.conllu'
with open(filename, 'r') as f:
    data = f.readlines()
i=0
sentences={}

for i in range(1,len(data)):
    line=data[i]
    if line[0]=='#':
        if line[2:9] == "sent_id":
            sentence_id = line[12:]
            
        if line[2:6] == "text":
            sentence =line[9:]
            sentences[sentence_id[:-1]] =sentence[:-1]
        
#print(sentences)
i = 0
attention={}
for sent in sentences :
    i+=1
    if i > 5: break
    tokens_sent = sentences[sent]
    tokens = tokenizer.tokenize(tokens_sent)
    #print(tokens_sent, tokens)
    tokens_a_delim = ['[CLS]'] + tokens + ['[SEP]']
    token_ids =  tokenizer.convert_tokens_to_ids(tokens_a_delim)
    #print(token_ids, len(token_ids))
    tokens_tensor = torch.tensor([token_ids])
    token_type_tensor = torch.LongTensor([[0] * len(tokens_a_delim)])
    #print(token_type_tensor)
    _, _, attn_data_list =  model(tokens_tensor, token_type_ids=token_type_tensor)
    attn_tensor = torch.stack([attn_data['attn_probs'] for attn_data in attn_data_list])
    attention[sent] = attn_tensor.data.numpy()


In [5]:

L=len(sentences)

sent_id = []
attentions = []
for idx in attention:
    sent_id.append(idx)
    attentions.append(attention[idx])
    print("att: ", len(attention[idx].shape))
    

#f= h5py.File('bert_attn.h5','w')
#dt = h5py.special_dtype(vlen=np.dtype('float64','float64'))
#dataset = f.create_dataset('vlen',(L,12,1,12,), dtype=dt)
#dataset.value
#for i in range(len(attentions)):
#    print("attention i: ", attention[i])
#    #dataset[i]=attentions[i]
#dataset.value
#f.close()    

att:  5
att:  5
att:  5
att:  5
att:  5


In [9]:
for i in range(len(attentions)):
    print("attention i: ", attentions[i].shape)

attention i:  (12, 1, 12, 40, 40)
attention i:  (12, 1, 12, 20, 20)
attention i:  (12, 1, 12, 41, 41)
attention i:  (12, 1, 12, 42, 42)
attention i:  (12, 1, 12, 15, 15)


In [6]:
f= h5py.File('bert_attn.h5','w')
dt = h5py.special_dtype(vlen=np.dtype('float64','float64'))
dataset = f.create_dataset('vlen',(L,12,1,12,), dtype=dt)
dataset.value
for i in range(len(attentions)):
    print("attention i: ", attention[i])
    #dataset[i]=attentions[i]
dataset.value
f.close()    

/home/pmh330/miniconda3/envs/jiant/lib/python3.6/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


KeyError: 0